In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="quick-starts-ws-140258")


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RKFR4E2FH to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-140258
Azure region: southcentralus
Subscription id: 976ee174-3882-4721-b90a-b5fef6b72f24
Resource group: aml-quickstarts-140258


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for the cluster
cpu_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Creating a new compute cluster...
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-12T06:14:45.994000+00:00', 'errors': None, 'creationTime': '2021-03-12T06:14:42.337392+00:00', 'modifiedTime': '2021-03-12T06:14:58.197650+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [8]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,quniform
import os

# Specify parameter sampler
parameter_space={
    '--C':uniform(0.5, 2),
    '--max_iter':quniform(50, 150, 10)}
ps =  RandomParameterSampling(parameter_space, properties=None) ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

sd='.'
path='train.py'

# Create a SKLearn estimator for use with train.py
est =SKLearn(source_directory=sd, compute_target=compute_target, vm_size='STANDARD_D2_V2', vm_priority=None, 
             entry_script=path, script_params={'--C':1.0,'--max_iter':100.0}, use_docker=True, custom_docker_image=None, 
             image_registry_details=None, user_managed=False, conda_packages=None, pip_packages=None, 
             conda_dependencies_file_path=None, pip_requirements_file_path=None, conda_dependencies_file=None, 
             pip_requirements_file=None, environment_variables=None, environment_definition=None, inputs=None, 
             shm_size=None, resume_from=None, max_run_duration_seconds=None, framework_version=None,
             _enable_optimized_mode=False, _disable_validation=True, _show_lint_warnings=False,
             _show_package_warnings=False) ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, primary_metric_name='Accuracy', primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                     max_total_runs=40, max_concurrent_runs=None, max_duration_minutes=10080, 
                                     policy=policy, estimator=est, run_config=None, resume_from=None,
                                     resume_child_runs=None, pipeline=None)### YOUR CODE HERE ###

In [9]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
run=exp.submit(hyperdrive_config, tags=None)
RunDetails(run).show()

The same input parameter(s) are specified in estimator/run_config script params and HyperDrive parameter space. HyperDrive parameter space definition will override these duplicate entries. ['--C', '--max_iter'] is the list of overridden parameter(s).


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [14]:
import joblib
# Get your best run and save the model from that run.
best_run=run.get_best_run_by_primary_metric()

ac=best_run.get_metrics()['Accuracy']
print('Best Accuracy we got:', ac)
print(best_run.get_details()['runDefinition']['arguments'])



Best Accuracy we got: 0.9119932022335518
['--C', '0.8347175441852543', '--max_iter', '120']


In [15]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path, validate=True, include_path=False, infer_column_types=True, 
                                                set_column_types=None, separator=',', 
                                                header=True, partition_format=None, support_multi_line=False, 
                                                empty_as_string=False)

In [19]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)### YOUR DATA OBJECT HERE ###)

data = pd.concat([x,y], axis=1)
#x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=32)

In [30]:
data.columns

Index(['age', 'marital', 'default', 'housing', 'loan', 'month', 'day_of_week',
       'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'contact_cellular', 'contact_telephone', 'education_basic.4y',
       'education_basic.6y', 'education_basic.9y', 'education_high.school',
       'education_illiterate', 'education_professional.course',
       'education_university.degree', 'education_unknown', 'y'],
      dtype='object')

In [40]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric='accuracy',
    training_data=data,
    label_column_name='y',
   # compute_target=compute_target,
    n_cross_validations=4)

In [41]:
# Submit your automl run

### YOUR CODE HERE ###
run = exp.submit(config=automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_f2c26ff6-406b-451a-8cf5-cf87c0186b46

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/Auto

In [42]:
# Retrieve and save your best automl model.
best_run, automl_model = run.get_output()
joblib.dump(automl_model, 'automl_model.joblib')
### YOUR CODE HERE ###

['automl_model.joblib']

In [43]:
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

